In [ ]:
# from sklearn.preprocessing import tfidf
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (8,716 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 144628 files and d

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data preprocessing

## 보습학원 (사설 교육 기관) 위치데이터
- 격자별 분석을 위한 좌표 추출

In [ ]:
academy = pd.read_csv('/content/drive/My Drive/초등보습학원주소.csv')

In [ ]:
academy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4989 entries, 0 to 4988
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   학원명     4989 non-null   object 
 1   학원주소    4989 non-null   object 
 2   정리주소    4989 non-null   object 
 3   bcode   4980 non-null   float64
 4   bname   4980 non-null   object 
 5   hcode   4978 non-null   float64
 6   hname   4978 non-null   object 
 7   latlng  4980 non-null   object 
dtypes: float64(2), object(6)
memory usage: 311.9+ KB


In [ ]:
gu = []
for i in academy['학원주소']:
  gu.append(i[6:10].strip())
academy['시군구명'] = gu

In [ ]:
academy['hcode'] = academy['hcode'].astype('str')
academy['bcode'] = academy['hcode'].astype('str')

In [ ]:
academy['hcode'] = academy['hcode'].str[:-2]
# academy['bcode'] = academy['hcode'].astype('str')

In [ ]:
academy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4989 entries, 0 to 4988
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   학원명     4989 non-null   object
 1   학원주소    4989 non-null   object
 2   정리주소    4989 non-null   object
 3   bcode   4989 non-null   object
 4   bname   4980 non-null   object
 5   hcode   4989 non-null   object
 6   hname   4978 non-null   object
 7   latlng  4980 non-null   object
 8   시군구명    4989 non-null   object
dtypes: object(9)
memory usage: 350.9+ KB


In [ ]:
academy

,학원명,학원주소,정리주소,bcode,bname,hcode,hname,latlng,시군구명
0,예맥보습학원,서울특별시 중랑구 동일로151길 30 2층3층 (묵동),서울특별시 중랑구 동일로151길 30,1126063000.0,묵동,1126063000,묵제2동,"37.6081791,127.0763503",중랑구
1,세진보습학원,서울특별시 중랑구 공릉로 58 2층4층일부 (묵동),서울특별시 중랑구 공릉로 58,1126062000.0,묵동,1126062000,묵제1동,"37.6158149,127.0792616",중랑구
2,대찬하브루타학원,"서울특별시 동대문구 전농로 120 , 3층 (전농동)",서울특별시 동대문구 전농로 120,1123057000.0,전농동,1123057000,전농제2동,"37.5756981,127.0578119",동대문구
3,장안로드맵수학학원,"서울특별시 동대문구 한천로 124 , 2층 202호 (장안동)",서울특별시 동대문구 한천로 124,1123065000.0,장안동,1123065000,장안제1동,"37.5721033,127.0661144",동대문구
4,더하이스트수학학원,"서울특별시 동대문구 사가정로 190 전농우성아파트 상가4층7,8,9호 (전농동)",서울특별시 동대문구 사가정로 190 전농우성아파트,1123057000.0,전농동,1123057000,전농제2동,"37.5783400,127.0660570",동대문구
...,...,...,...,...,...,...,...,...,...
4984,헤럴드스쿨어학원,"서울특별시 성북구 종암로 120 , 2층 일부 (종암동)",서울특별시 성북구 종암로 120,1129070500.0,종암동,1129070500,종암동,"37.6009223,127.0338772",성북구
4985,청담강북어학원,"서울특별시 강북구 삼양로27길 35-21 , 5층 501,502,504,505호 (...",서울특별시 강북구 삼양로27길 35-21,1130557500.0,미아동,1130557500,삼각산동,"37.6179397,127.0187979",강북구
4986,EIE(이아이이)어학원,"서울특별시 강북구 도봉로 189 , 4층,5층 (미아동)",서울특별시 강북구 도봉로 189,1130553500.0,미아동,1130553500,미아동,"37.6256857,127.0261652",강북구
4987,성북메가스터디학원,서울특별시 성북구 아리랑로 17 민국빌딩 1-5층 (동소문동6가),서울특별시 성북구 아리랑로 17 민국빌딩,1129057500.0,동소문동6가,1129057500,동선동,"37.5944241,127.0159501",성북구


In [ ]:
academy['구동']= academy['시군구명'] +academy['hname']
std = academy[['구동','시군구명','hcode','hname']]
# std['hcode'][0][:-2]

In [ ]:
len(std['hcode'].unique())

401

3486     서울특별시 강남구 언주로 21 , 205-2호,210호,213-1호 (개포동,개포시영아파트상가)

 3660     서울특별시 서초구 서초대로 61 , 2층 (방배동)

In [ ]:
academy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4989 entries, 0 to 4988
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   학원명     4989 non-null   object
 1   학원주소    4989 non-null   object
 2   정리주소    4989 non-null   object
 3   bcode   4989 non-null   object
 4   bname   4980 non-null   object
 5   hcode   4989 non-null   object
 6   hname   4978 non-null   object
 7   latlng  4980 non-null   object
 8   시군구명    4989 non-null   object
 9   구동      4978 non-null   object
dtypes: object(10)
memory usage: 389.9+ KB


In [ ]:
academy.to_excel('chemdqhtmqgkrdnjs.xlsx',encoding='cp949')

In [ ]:
aca = pd.read_excel('/content/drive/My Drive/chemdqhtmqgkrdnjs.xlsx')

In [ ]:
aca

,Unnamed: 0,학원명,학원주소,정리주소,bcode,bname,hcode,hname,latlng
0,0,예맥보습학원,서울특별시 중랑구 동일로151길 30 2층3층 (묵동),서울특별시 중랑구 동일로151길 30,1.126010e+09,묵동,1126063000,묵제2동,"37.6081791,127.0763503"
1,1,세진보습학원,서울특별시 중랑구 공릉로 58 2층4층일부 (묵동),서울특별시 중랑구 공릉로 58,1.126010e+09,묵동,1126062000,묵제1동,"37.6158149,127.0792616"
2,2,대찬하브루타학원,"서울특별시 동대문구 전농로 120 , 3층 (전농동)",서울특별시 동대문구 전농로 120,1.123010e+09,전농동,1123057000,전농제2동,"37.5756981,127.0578119"
3,3,장안로드맵수학학원,"서울특별시 동대문구 한천로 124 , 2층 202호 (장안동)",서울특별시 동대문구 한천로 124,1.123011e+09,장안동,1123065000,장안제1동,"37.5721033,127.0661144"
4,4,더하이스트수학학원,"서울특별시 동대문구 사가정로 190 전농우성아파트 상가4층7,8,9호 (전농동)",서울특별시 동대문구 사가정로 190 전농우성아파트,1.123010e+09,전농동,1123057000,전농제2동,"37.5783400,127.0660570"
...,...,...,...,...,...,...,...,...,...
4978,4984,헤럴드스쿨어학원,"서울특별시 성북구 종암로 120 , 2층 일부 (종암동)",서울특별시 성북구 종암로 120,1.129014e+09,종암동,1129070500,종암동,"37.6009223,127.0338772"
4979,4985,청담강북어학원,"서울특별시 강북구 삼양로27길 35-21 , 5층 501,502,504,505호 (...",서울특별시 강북구 삼양로27길 35-21,1.130510e+09,미아동,1130557500,삼각산동,"37.6179397,127.0187979"
4980,4986,EIE(이아이이)어학원,"서울특별시 강북구 도봉로 189 , 4층,5층 (미아동)",서울특별시 강북구 도봉로 189,1.130510e+09,미아동,1130553500,미아동,"37.6256857,127.0261652"
4981,4987,성북메가스터디학원,서울특별시 성북구 아리랑로 17 민국빌딩 1-5층 (동소문동6가),서울특별시 성북구 아리랑로 17 민국빌딩,1.129011e+09,동소문동6가,1129057500,동선동,"37.5944241,127.0159501"


In [ ]:
aca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4983 entries, 0 to 4982
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  4983 non-null   int64  
 1   학원명         4983 non-null   object 
 2   학원주소        4983 non-null   object 
 3   정리주소        4983 non-null   object 
 4   bcode       4982 non-null   float64
 5   bname       4983 non-null   object 
 6   hcode       4983 non-null   int64  
 7   hname       4983 non-null   object 
 8   latlng      4983 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 350.5+ KB


In [ ]:
aca['hcode'] = aca['hcode'].astype('str')
aca['bcode'] = aca['hcode'].astype('str')

In [ ]:
aca

,Unnamed: 0,학원명,학원주소,정리주소,bcode,bname,hcode,hname,latlng
0,0,예맥보습학원,서울특별시 중랑구 동일로151길 30 2층3층 (묵동),서울특별시 중랑구 동일로151길 30,1126063000,묵동,1126063000,묵제2동,"37.6081791,127.0763503"
1,1,세진보습학원,서울특별시 중랑구 공릉로 58 2층4층일부 (묵동),서울특별시 중랑구 공릉로 58,1126062000,묵동,1126062000,묵제1동,"37.6158149,127.0792616"
2,2,대찬하브루타학원,"서울특별시 동대문구 전농로 120 , 3층 (전농동)",서울특별시 동대문구 전농로 120,1123057000,전농동,1123057000,전농제2동,"37.5756981,127.0578119"
3,3,장안로드맵수학학원,"서울특별시 동대문구 한천로 124 , 2층 202호 (장안동)",서울특별시 동대문구 한천로 124,1123065000,장안동,1123065000,장안제1동,"37.5721033,127.0661144"
4,4,더하이스트수학학원,"서울특별시 동대문구 사가정로 190 전농우성아파트 상가4층7,8,9호 (전농동)",서울특별시 동대문구 사가정로 190 전농우성아파트,1123057000,전농동,1123057000,전농제2동,"37.5783400,127.0660570"
...,...,...,...,...,...,...,...,...,...
4978,4984,헤럴드스쿨어학원,"서울특별시 성북구 종암로 120 , 2층 일부 (종암동)",서울특별시 성북구 종암로 120,1129070500,종암동,1129070500,종암동,"37.6009223,127.0338772"
4979,4985,청담강북어학원,"서울특별시 강북구 삼양로27길 35-21 , 5층 501,502,504,505호 (...",서울특별시 강북구 삼양로27길 35-21,1130557500,미아동,1130557500,삼각산동,"37.6179397,127.0187979"
4980,4986,EIE(이아이이)어학원,"서울특별시 강북구 도봉로 189 , 4층,5층 (미아동)",서울특별시 강북구 도봉로 189,1130553500,미아동,1130553500,미아동,"37.6256857,127.0261652"
4981,4987,성북메가스터디학원,서울특별시 성북구 아리랑로 17 민국빌딩 1-5층 (동소문동6가),서울특별시 성북구 아리랑로 17 민국빌딩,1129057500,동소문동6가,1129057500,동선동,"37.5944241,127.0159501"


In [ ]:
str(aca['latlng']).split(',')[0]

'0       37.6081791'

In [ ]:
type(aca['latlng'][0].split(',')[0])

str

In [ ]:
aca['Lat'], aca['Lon'] = aca['latlng'].str.split(',', 2).str

In [ ]:
aca.columns

Index(['Unnamed: 0', '학원명', '학원주소', '정리주소', 'bcode', 'bname', 'hcode', 'hname',
       'latlng', 'Lat', 'Lon'],
      dtype='object')

In [ ]:
aca_df =aca[[ '학원명', '학원주소', 'hcode', 'hname', 'Lat', 'Lon']]
# aca_df.to_pickle('초등보습학원_좌표.pkl')

In [ ]:
df_aca = aca[['hcode','학원명','학원주소','hname','Lat','Lon']]
# df_aca[df_aca['hname']=='신사동']
df_aca

,hcode,학원명,학원주소,hname,Lat,Lon
0,1126063000,예맥보습학원,서울특별시 중랑구 동일로151길 30 2층3층 (묵동),묵제2동,37.6081791,127.0763503
1,1126062000,세진보습학원,서울특별시 중랑구 공릉로 58 2층4층일부 (묵동),묵제1동,37.6158149,127.0792616
2,1123057000,대찬하브루타학원,"서울특별시 동대문구 전농로 120 , 3층 (전농동)",전농제2동,37.5756981,127.0578119
3,1123065000,장안로드맵수학학원,"서울특별시 동대문구 한천로 124 , 2층 202호 (장안동)",장안제1동,37.5721033,127.0661144
4,1123057000,더하이스트수학학원,"서울특별시 동대문구 사가정로 190 전농우성아파트 상가4층7,8,9호 (전농동)",전농제2동,37.5783400,127.0660570
...,...,...,...,...,...,...
4978,1129070500,헤럴드스쿨어학원,"서울특별시 성북구 종암로 120 , 2층 일부 (종암동)",종암동,37.6009223,127.0338772
4979,1130557500,청담강북어학원,"서울특별시 강북구 삼양로27길 35-21 , 5층 501,502,504,505호 (...",삼각산동,37.6179397,127.0187979
4980,1130553500,EIE(이아이이)어학원,"서울특별시 강북구 도봉로 189 , 4층,5층 (미아동)",미아동,37.6256857,127.0261652
4981,1129057500,성북메가스터디학원,서울특별시 성북구 아리랑로 17 민국빌딩 1-5층 (동소문동6가),동선동,37.5944241,127.0159501


관악구 신사동 : 1162068500	
강님구 신사동 : 1168051000

In [ ]:
import pickle

## 행정동별 거주인구 데이터 전처리
- 연령 / 성별 / 나이(5세단위) 구분


In [ ]:
stand = pd.read_pickle('/content/drive/My Drive/project/행정동별_거주인구_성별_연령.pickle')


In [ ]:
stand

,ADMI_CD,ADMI_NM,5559세_2,5559세_1,5054세_1,5054세_2,4549세_1,4549세_2,4044세_1,4044세_2,3539세_1,3539세_2,3034세_1,3034세_2,2529세_1,2529세_2,2024세_1,2024세_2,1519세_1,1519세_2,1014세_1,1014세_2,0509세_2,0509세_1
1725910,1117051000,후암동,757,709,722,763,745,782,601,653,737,717,731,661,783,748,540,524,422,330,310,270,266,299
1725912,1159053000,상도1동,1691,1439,1535,1723,1716,1678,1616,1657,1580,1713,1632,1533,2631,2201,2346,2649,977,908,792,773,839,880
1725913,1111070000,숭인1동,288,305,254,293,292,258,216,202,238,209,201,197,228,225,184,179,102,114,85,92,94,98
1725917,1111061500,종로1.2.3.4가동,251,549,431,164,341,218,242,157,255,207,252,191,308,212,142,195,59,66,58,41,46,54
1725919,1117057000,원효로2동,547,586,569,634,633,640,530,576,569,530,499,489,501,530,385,471,292,440,230,341,267,271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728432,1153059500,가리봉동,425,592,589,369,411,338,284,210,332,220,379,288,498,368,257,283,96,99,72,56,65,74
1728436,1156055000,당산1동,783,774,783,756,799,718,754,701,1064,881,1149,1134,1087,1256,552,708,300,301,220,212,274,253
1728438,1141072000,북가좌2동,1486,1272,1390,1399,1383,1438,1147,1191,1210,1144,1073,1017,1373,1310,1116,1110,768,726,544,525,477,547
1728439,1165062000,방배3동,1024,856,838,916,852,1027,686,786,738,814,716,781,805,720,714,688,666,547,393,460,368,396


In [ ]:
stand.columns

Index(['ADMI_CD', 'ADMI_NM', '5559세_2', '5559세_1', '5054세_1', '5054세_2',
       '4549세_1', '4549세_2', '4044세_1', '4044세_2', '3539세_1', '3539세_2',
       '3034세_1', '3034세_2', '2529세_1', '2529세_2', '2024세_1', '2024세_2',
       '1519세_1', '1519세_2', '1014세_1', '1014세_2', '0509세_2', '0509세_1'],
      dtype='object')

In [ ]:
stand['5-14세'] = stand['1014세_1']+stand['1014세_2']+stand['0509세_2']+stand['0509세_1']
stand

,ADMI_CD,ADMI_NM,5559세_2,5559세_1,5054세_1,5054세_2,4549세_1,4549세_2,4044세_1,4044세_2,3539세_1,3539세_2,3034세_1,3034세_2,2529세_1,2529세_2,2024세_1,2024세_2,1519세_1,1519세_2,1014세_1,1014세_2,0509세_2,0509세_1,5-14세
1725910,1117051000,후암동,757,709,722,763,745,782,601,653,737,717,731,661,783,748,540,524,422,330,310,270,266,299,1145
1725912,1159053000,상도1동,1691,1439,1535,1723,1716,1678,1616,1657,1580,1713,1632,1533,2631,2201,2346,2649,977,908,792,773,839,880,3284
1725913,1111070000,숭인1동,288,305,254,293,292,258,216,202,238,209,201,197,228,225,184,179,102,114,85,92,94,98,369
1725917,1111061500,종로1.2.3.4가동,251,549,431,164,341,218,242,157,255,207,252,191,308,212,142,195,59,66,58,41,46,54,199
1725919,1117057000,원효로2동,547,586,569,634,633,640,530,576,569,530,499,489,501,530,385,471,292,440,230,341,267,271,1109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728432,1153059500,가리봉동,425,592,589,369,411,338,284,210,332,220,379,288,498,368,257,283,96,99,72,56,65,74,267
1728436,1156055000,당산1동,783,774,783,756,799,718,754,701,1064,881,1149,1134,1087,1256,552,708,300,301,220,212,274,253,959
1728438,1141072000,북가좌2동,1486,1272,1390,1399,1383,1438,1147,1191,1210,1144,1073,1017,1373,1310,1116,1110,768,726,544,525,477,547,2093
1728439,1165062000,방배3동,1024,856,838,916,852,1027,686,786,738,814,716,781,805,720,714,688,666,547,393,460,368,396,1617


In [ ]:
 std_ = stand[['ADMI_NM','ADMI_CD','5-14세']]
 std_.columns = ['행정구역','행정동코드','5-14세']
#  std.to_excel('기준.xlsx')

### 신사동 처리 
- 행정동끼리 이름이 같은 경우 유의

In [ ]:
std_[std_['행정구역']=='신사동']
# 관악구 신사동 : 1162068500	
# 강님구 신사동 : 1168051000

,행정구역,행정동코드,5-14세
1727712,신사동,1168051000,1422
1727837,신사동,1162068500,939


In [ ]:
# std[std['hcode']=='1168051000']
# 1162068500
# std[std['hcode']=='1162068500']

std.drop_duplicates(subset=['hcode'],keep='last',inplace=True)
std

,구동,시군구명,hcode,hname
68,중랑구상봉제1동,중랑구,1126058000,상봉제1동
159,중랑구중화제2동,중랑구,1126061000,중화제2동
195,중랑구면목제5동,중랑구,1126055000,면목제5동
224,동대문구전농제2동,동대문구,1123057000,전농제2동
228,중랑구묵제1동,중랑구,1126062000,묵제1동
...,...,...,...,...
4984,성북구종암동,성북구,1129070500,종암동
4985,강북구삼각산동,강북구,1130557500,삼각산동
4986,강북구미아동,강북구,1130553500,미아동
4987,성북구동선동,성북구,1129057500,동선동


In [ ]:
std_

,행정구역,행정동코드,5-14세
1725910,후암동,1117051000,1145
1725912,상도1동,1159053000,3284
1725913,숭인1동,1111070000,369
1725917,종로1.2.3.4가동,1111061500,199
1725919,원효로2동,1117057000,1109
...,...,...,...
1728432,가리봉동,1153059500,267
1728436,당산1동,1156055000,959
1728438,북가좌2동,1141072000,2093
1728439,방배3동,1165062000,1617


 행정구역 시군구명과 행정동 코드 기준 모두 합친 standard

In [ ]:
standard = pd.merge(std_.set_index('행정동코드'),std.set_index('hcode'),left_index=True,right_index=True)
# standard[standard['행정구역']=='신사동']
standard

,행정구역,5-14세,구동,시군구명,hname
1117051000,후암동,1145,용산구후암동,용산구,후암동
1159053000,상도1동,3284,동작구상도제1동,동작구,상도제1동
1111070000,숭인1동,369,종로구숭인제1동,종로구,숭인제1동
1111061500,종로1.2.3.4가동,199,종로구종로1.2.3.4가동,종로구,종로1.2.3.4가동
1117057000,원효로2동,1109,용산구원효로제2동,용산구,원효로제2동
...,...,...,...,...,...
1153059500,가리봉동,267,구로구가리봉동,구로구,가리봉동
1156055000,당산1동,959,영등포구당산제1동,영등포구,당산제1동
1141072000,북가좌2동,2093,서대문구북가좌제2동,서대문구,북가좌제2동
1165062000,방배3동,1617,서초구방배3동,서초구,방배3동


In [ ]:
len(df_aca['hcode'].unique())

400

In [ ]:
aca_dong = df_aca.groupby('hcode').count()
aca_cnt = aca_dong[['hname']]

In [ ]:
aca_cnt.reset_index(inplace=True)

In [ ]:
aca_cnt.columns = ['행정동코드','보습학원개수']
aca_cnt

,행정동코드,보습학원개수
0,1111051500,1
1,1111053000,2
2,1111055000,7
3,1111056000,5
4,1111057000,3
...,...,...
395,1174064000,18
396,1174065000,6
397,1174066000,30
398,1174068500,29


## 행정동별 예술학원 (사설 교육 기관)
- 보습학원과 달리 위치 데이터 없음
- 사설 교육기간 중요도 산출에만 사용되고 후에 격자 추출에는 반영하지 않음

In [ ]:
art = pd.read_pickle('/content/drive/My Drive/행정동별_예술학원.pkl')
art.to_excel('음악학원.xlsx')

In [ ]:
art.columns

Index(['행정구역', '생활밀접업종', '2018년 2분기', '2019년 2분기', '2020년 2분기', '시군구', '구동'], dtype='object')

In [ ]:
a = art[['행정구역','시군구','2020년 2분기']]
a

,행정구역,시군구,2020년 2분기
6,청운효자동,종로구,15
8,사직동,종로구,13
10,삼청동,종로구,1
12,부암동,종로구,8
14,평창동,종로구,4
...,...,...,...
892,성내2동,강동구,16
894,성내3동,강동구,22
896,길동,강동구,40
898,둔촌1동,강동구,2


In [ ]:
std_

,행정구역,행정동코드,5-14세
1725910,후암동,1117051000,1145
1725912,상도1동,1159053000,3284
1725913,숭인1동,1111070000,369
1725917,종로1.2.3.4가동,1111061500,199
1725919,원효로2동,1117057000,1109
...,...,...,...
1728432,가리봉동,1153059500,267
1728436,당산1동,1156055000,959
1728438,북가좌2동,1141072000,2093
1728439,방배3동,1165062000,1617


In [ ]:
art_new = pd.merge(std_,a,how='outer',on='행정구역')
# art_new.drop_duplicates(subset=['행정구역'],keep='last',inplace=True)
art_new

,행정구역,행정동코드,5-14세,시군구,2020년 2분기
0,후암동,1117051000,1145,용산구,18
1,상도1동,1159053000,3284,동작구,39
2,숭인1동,1111070000,369,종로구,2
3,종로1.2.3.4가동,1111061500,199,종로구,32
4,원효로2동,1117057000,1109,용산구,12
...,...,...,...,...,...
421,가리봉동,1153059500,267,구로구,4
422,당산1동,1156055000,959,영등포구,13
423,북가좌2동,1141072000,2093,서대문구,28
424,방배3동,1165062000,1617,서초구,26


In [ ]:
art_new[art_new['행정구역']=='신사동']
# 관악구 신사동 : 1162068500	
# 강님구 신사동 : 1168051000

,행정구역,행정동코드,5-14세,시군구,2020년 2분기
281,신사동,1168051000,1422,관악구,15
282,신사동,1168051000,1422,강남구,67
283,신사동,1162068500,939,관악구,15
284,신사동,1162068500,939,강남구,67


In [ ]:
art_new.drop([281,284],inplace=True)

In [ ]:
art_new[art_new['행정구역']=='신사동']

,행정구역,행정동코드,5-14세,시군구,2020년 2분기
282,신사동,1168051000,1422,강남구,67
283,신사동,1162068500,939,관악구,15


In [ ]:
# art_new.columns = ['행정구역','행정동코드','시군구','예술학원']
art_new

,행정구역,행정동코드,5-14세,시군구,2020년 2분기
0,후암동,1117051000,1145,용산구,18
1,상도1동,1159053000,3284,동작구,39
2,숭인1동,1111070000,369,종로구,2
3,종로1.2.3.4가동,1111061500,199,종로구,32
4,원효로2동,1117057000,1109,용산구,12
...,...,...,...,...,...
421,가리봉동,1153059500,267,구로구,4
422,당산1동,1156055000,959,영등포구,13
423,북가좌2동,1141072000,2093,서대문구,28
424,방배3동,1165062000,1617,서초구,26


In [ ]:
aca_cnt

,행정동코드,보습학원개수
0,1111051500,1
1,1111053000,2
2,1111055000,7
3,1111056000,5
4,1111057000,3
...,...,...
395,1174064000,18
396,1174065000,6
397,1174066000,30
398,1174068500,29


In [ ]:
all = pd.merge(art_new,aca_cnt,how='left',on=['행정동코드'])
all

,행정구역,행정동코드,5-14세,시군구,2020년 2분기,보습학원개수
0,후암동,1117051000,1145,용산구,18,6.0
1,상도1동,1159053000,3284,동작구,39,19.0
2,숭인1동,1111070000,369,종로구,2,1.0
3,종로1.2.3.4가동,1111061500,199,종로구,32,4.0
4,원효로2동,1117057000,1109,용산구,12,3.0
...,...,...,...,...,...,...
419,가리봉동,1153059500,267,구로구,4,1.0
420,당산1동,1156055000,959,영등포구,13,1.0
421,북가좌2동,1141072000,2093,서대문구,28,32.0
422,방배3동,1165062000,1617,서초구,26,6.0


In [ ]:
all.fillna(0,inplace=True)

In [ ]:
all.columns
all.columns = ['행정구역', '행정동코드', '5-14세', '시군구', '예술학원', '보습학원개수']

In [ ]:
all['총학원수'] = all['보습학원개수'] + all['예술학원']
all

,행정구역,행정동코드,5-14세,시군구,예술학원,보습학원개수,총학원수
0,후암동,1117051000,1145,용산구,18,6.0,24.0
1,상도1동,1159053000,3284,동작구,39,19.0,58.0
2,숭인1동,1111070000,369,종로구,2,1.0,3.0
3,종로1.2.3.4가동,1111061500,199,종로구,32,4.0,36.0
4,원효로2동,1117057000,1109,용산구,12,3.0,15.0
...,...,...,...,...,...,...,...
419,가리봉동,1153059500,267,구로구,4,1.0,5.0
420,당산1동,1156055000,959,영등포구,13,1.0,14.0
421,북가좌2동,1141072000,2093,서대문구,28,32.0,60.0
422,방배3동,1165062000,1617,서초구,26,6.0,32.0


In [ ]:
all.to_pickle('/content/drive/My Drive/project/seoul_contest/행정동별_사설학원(일반보습,예술)12.pkl')


강북구 번2동을 제외한 모든 행정동에는 하나 이상의 학원이 있다 !

-> nan 값은 0 으로 치환 함

-> tfidf smoothing 할 필요 없음

In [ ]:
all[all['총학원수'] == 0]

,행정구역,행정동코드,시군구,예술학원,보습학원개수,총학원수
46,번2동,1130560300,강북구,0,0.0,0.0


# 중요도 산출

## 행정동별 사설학원 tfidf (단순빈도)


weighting schema|weight|설명
--|--|--
tf(double normalization 0.5)|<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/45badc1c70ec2caa00ed8c21ed75bd9f8d3e650c" />|=0.5 + 0.5(토큰빈도/문서내최빈토큰)
idf(inverse document frequency smooth)|<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/25f4d6690acaaef1f15f308d24f6f8a439de971d" />|=log(문서갯수/(1+토큰빈도)) + 1

Tf = 문서내 토크빈도 / 문서내 전체 토큰 갯수

In [ ]:
cnt = 0
for i in range(len(all['총학원수'])):
    if all['총학원수'][i] != 0:
        cnt +=all['총학원수'][i]
cnt

12825.0

In [ ]:
acnt = 0
for i in range(len(all['총학원수'])):
    if all['총학원수'][i] != 0:
        acnt +=1
acnt

423

In [ ]:
all.columns

Index(['행정구역', '행정동코드', '5-14세', '시군구', '예술학원', '보습학원개수', '총학원수'], dtype='object')

In [ ]:
tf = [(float(all['총학원수'][i] + 1)/all['5-14세'][i]) for i in range(len(all))]
idf = np.log(len(all) / (acnt))
tfidf = [x*idf for x in tf]

In [ ]:
all['학원tfidf'] = tfidf
# len(all.행정구역.unique())
all[all['행정구역']=='길음1동']

,행정구역,행정동코드,5-14세,시군구,예술학원,보습학원개수,총학원수,학원tfidf
283,길음1동,1129066000,5305,성북구,30,18.0,48.0,0.000022


In [ ]:
dolbom = pd.read_csv('/content/drive/My Drive/project/seoul_contest/행정동별_공적돌봄시설_필요지수_대체시설제외.csv')

## 행정동별 공적 돌봄 시설 tfidf

In [ ]:
dolbom['행정동코드']=dolbom['행정동코드'].astype(str)
dolbom['0514세']=dolbom['0514세'].astype(int)
dolbom['1014세']=dolbom['1014세'].astype(int)
dolbom['0509세']=dolbom['0509세'].astype(int)

In [ ]:
dolbom.columns
dolbom = dolbom[['행정동코드', '행정동','0514세', '시군구명','키움센터_개수', '초등돌봄교실_교실개수',
       '지역아동센터_개수', '방과후아카데미_개수', '총시설수' ]]

In [ ]:
cnt = 0
for i in range(len(dolbom['총시설수'])):
    if dolbom['총시설수'][i] != 0:
        cnt +=1
cnt

400

In [ ]:
# tf2 = [(float(dolbom['총시설수'][i] + 1)/dolbom['0514세'][i]) for i in range(len(dolbom))]
tf3 = [(float(max(dolbom['총시설수']) - dolbom['총시설수'][i] + 1)/dolbom['0514세'][i]) for i in range(len(dolbom))]
idf2 = np.log(len(dolbom) / (cnt+1))
# tfidf2 = [x*idf2 for x in tf2]
tfidf3 = [x*idf2 for x in tf3]
dolbom['시설tfidf'] = tfidf3
# dolbom['시설tfidf_max'] = tfidf3

In [ ]:
dolbom

,행정동코드,행정동,0514세,시군구명,키움센터_개수,초등돌봄교실_교실개수,지역아동센터_개수,방과후아카데미_개수,총시설수,시설tfidf
0,1165065100,양재1동,5145,서초구,1.0,21.0,2.0,0.0,24.0,0.000011
1,1153078000,오류제2동,4507,구로구,1.0,17.0,2.0,0.0,20.0,0.000062
2,1150063000,방화제1동,4970,강서구,1.0,12.0,3.0,0.0,16.0,0.000101
3,1138069000,진관동,7384,은평구,1.0,21.0,1.0,0.0,23.0,0.000015
4,1138062500,역촌동,3381,은평구,1.0,6.0,4.0,0.0,11.0,0.000231
...,...,...,...,...,...,...,...,...,...,...
419,1114057000,필동,180,중구,0.0,0.0,0.0,0.0,0.0,0.007746
420,1114059000,광희동,170,중구,0.0,0.0,0.0,0.0,0.0,0.008202
421,1111054000,삼청동,165,종로구,0.0,0.0,0.0,0.0,0.0,0.008450
422,1114060500,을지로동,44,중구,0.0,0.0,0.0,0.0,0.0,0.031689


## 인프라 지수 산출

In [ ]:
df = pd.merge(dolbom,all,on=['행정동코드'])
df = df[['행정동코드', '행정동', '시군구명', '5-14세', '키움센터_개수', '초등돌봄교실_교실개수', '지역아동센터_개수',
       '방과후아카데미_개수', '총시설수', '시설tfidf', '예술학원',
       '보습학원개수', '총학원수', '학원tfidf']]
df['인프라지수'] = df['시설tfidf']/df['학원tfidf']
df

,행정동코드,행정동,시군구명,5-14세,키움센터_개수,초등돌봄교실_교실개수,지역아동센터_개수,방과후아카데미_개수,총시설수,시설tfidf,예술학원,보습학원개수,총학원수,학원tfidf,인프라지수
0,1165065100,양재1동,서초구,5145,1.0,21.0,2.0,0.0,24.0,0.000011,35,17.0,52.0,0.000024,0.445650
1,1153078000,오류제2동,구로구,4507,1.0,17.0,2.0,0.0,20.0,0.000062,44,26.0,70.0,0.000037,1.663341
2,1150063000,방화제1동,강서구,4970,1.0,12.0,3.0,0.0,16.0,0.000101,42,47.0,89.0,0.000043,2.361944
3,1138069000,진관동,은평구,7384,1.0,21.0,1.0,0.0,23.0,0.000015,37,27.0,64.0,0.000021,0.726752
4,1138062500,역촌동,은평구,3381,1.0,6.0,4.0,0.0,11.0,0.000231,28,43.0,71.0,0.000050,4.592669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,1114057000,필동,중구,180,0.0,0.0,0.0,0.0,0.0,0.007746,3,0.0,3.0,0.000052,147.621518
420,1114059000,광희동,중구,170,0.0,0.0,0.0,0.0,0.0,0.008202,1,0.0,1.0,0.000028,295.243036
421,1111054000,삼청동,종로구,165,0.0,0.0,0.0,0.0,0.0,0.008450,1,0.0,1.0,0.000029,295.243036
422,1114060500,을지로동,중구,44,0.0,0.0,0.0,0.0,0.0,0.031689,2,0.0,2.0,0.000161,196.828690


In [ ]:
df[df['행정동']=='길음제1동']

,행정동코드,행정동,시군구명,5-14세,키움센터_개수,초등돌봄교실_교실개수,지역아동센터_개수,방과후아카데미_개수,총시설수,시설tfidf,예술학원,보습학원개수,총학원수,학원tfidf,인프라지수,인프라지수_mmscaler,학원지수_mmscaler,시설지수_mmscaler,학원100,시설100,학원percentile,시설percentile
15,1129066000,길음제1동,성북구,5305,0.0,13.0,1.0,0.0,14.0,0.000116,30,18.0,48.0,0.000022,5.302324,0.008574,0.029993,0.000827,0.218101,1.156442,2.0,1.0


### 인프라 지수 minmaxscaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
fitted = scaler.fit_transform(np.array(df['인프라지수']).reshape(-1,1))
output = pd.DataFrame(fitted)
df['인프라지수_mmscaler'] = output

### 인프라지수 percentile

In [ ]:
pt = np.percentile(df[['인프라지수']],[i*10 for i in range(1,10)]).tolist()
df['인프라지수percentile'] = 0
df.loc[(df['인프라지수'] <= pt[0]),'인프라지수percentile'] = '1'
df.loc[(df['인프라지수'] > pt[0]) & (df['인프라지수'] <= pt[1]),'인프라지수percentile'] = '2'
df.loc[(df['인프라지수'] > pt[1]) & (df['인프라지수'] <= pt[2]),'인프라지수percentile'] = '3'
df.loc[(df['인프라지수'] > pt[2]) & (df['인프라지수'] <= pt[3]),'인프라지수percentile'] = '4'
df.loc[(df['인프라지수'] > pt[3]) & (df['인프라지수'] <= pt[4]),'인프라지수percentile'] = '5'
df.loc[(df['인프라지수'] > pt[4]) & (df['인프라지수'] <= pt[5]),'인프라지수percentile'] = '6'
df.loc[(df['인프라지수'] > pt[5]) & (df['인프라지수'] <= pt[6]),'인프라지수percentile'] = '7'
df.loc[(df['인프라지수'] > pt[6]) & (df['인프라지수'] <= pt[7]),'인프라지수percentile'] = '8'
df.loc[(df['인프라지수'] > pt[7]) & (df['인프라지수'] <= pt[8]),'인프라지수percentile'] = '9'
df.loc[(df['인프라지수'] > pt[8]),'인프라지수percentile'] = '10'

In [ ]:
df.sort_values(by='인프라지수',ascending=False)
df.to_pickle('인프라지수1109.pkl')

In [ ]:
with open('/content/drive/My Drive/인프라지수1109.pkl', 'rb') as f:  
  df = pickle.load(f)

In [ ]:
df.sort_values(by='인프라지수',ascending=False)

,행정동코드,행정동,시군구명,5-14세,키움센터_개수,초등돌봄교실_교실개수,지역아동센터_개수,방과후아카데미_개수,총시설수,시설tfidf,예술학원,보습학원개수,총학원수,학원tfidf,인프라지수,인프라지수_mmscaler,인프라지수percentile
290,1130560300,번2동,강북구,918,0.0,0.0,1.0,0.0,1.0,0.001458,0,0.0,0.0,0.000003,566.866628,1.000000,10
420,1114059000,광희동,중구,170,0.0,0.0,0.0,0.0,0.0,0.008202,1,0.0,1.0,0.000028,295.243036,0.520456,10
418,1114054000,회현동,중구,224,0.0,0.0,0.0,0.0,0.0,0.006225,1,0.0,1.0,0.000021,295.243036,0.520456,10
417,1111071000,숭인제2동,종로구,297,0.0,0.0,0.0,0.0,0.0,0.004695,1,0.0,1.0,0.000016,295.243036,0.520456,10
421,1111054000,삼청동,종로구,165,0.0,0.0,0.0,0.0,0.0,0.008450,1,0.0,1.0,0.000029,295.243036,0.520456,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,1159066000,대방동,동작구,3487,0.0,17.0,1.0,0.0,18.0,0.000112,27,45.0,72.0,0.000049,2.264878,0.003212,1
1,1153078000,오류제2동,구로구,4507,1.0,17.0,2.0,0.0,20.0,0.000062,44,26.0,70.0,0.000037,1.663341,0.002150,1
7,1168070000,세곡동,강남구,5610,1.0,21.0,1.0,0.0,23.0,0.000020,26,15.0,41.0,0.000018,1.124735,0.001199,1
3,1138069000,진관동,은평구,7384,1.0,21.0,1.0,0.0,23.0,0.000015,37,27.0,64.0,0.000021,0.726752,0.000496,1


In [ ]:
!pip install folium

In [ ]:
import folium

In [ ]:
geo_data = 'data/seoul-dong.geojson'

In [ ]:
df.columns

Index(['행정동코드', '행정동', '시군구명', '5-14세', '키움센터_개수', '초등돌봄교실_교실개수', '지역아동센터_개수',
       '방과후아카데미_개수', '총시설수', '시설tfidf', '예술학원', '보습학원개수', '총학원수', '학원tfidf',
       '인프라지수', '인프라지수_mmscaler', '인프라지수percentile'],
      dtype='object')

In [ ]:
!pip install geopandas

     |████████████████████████████████| 972kB 2.8MB/s 
     |████████████████████████████████| 14.8MB 326kB/s 
     |████████████████████████████████| 6.5MB 33.7MB/s 


In [ ]:
import folium 
import geopandas as gpd
import math
import fiona
# import utm
from shapely.geometry import Point, Polygon

import folium
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from haversine import haversine
from tqdm.notebook import tqdm
%matplotlib inline
import platform
import pickle
import re
platform.system()
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'

from IPython.display import display #print가 아닌 display()로 연속 출력
from IPython.display import HTML #출력 결과를 HTML로 생성

In [ ]:
df.columns

Index(['행정동코드', '행정동', '시군구명', '5-14세', '키움센터_개수', '초등돌봄교실_교실개수', '지역아동센터_개수',
       '방과후아카데미_개수', '총시설수', '시설tfidf', '예술학원', '보습학원개수', '총학원수', '학원tfidf',
       '인프라지수', '인프라지수_mmscaler'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
fitted1 = scaler.fit_transform(np.array(df['시설tfidf']).reshape(-1,1))
fitted2 = scaler.fit_transform(np.array(df['학원tfidf']).reshape(-1,1))
output1 = pd.DataFrame(fitted1)
output2 = pd.DataFrame(fitted2)
df['학원지수_mmscaler'] = output2
df['시설지수_mmscaler'] = output1
df['학원100']=df['학원tfidf']*10000
df['시설100']=df['시설tfidf']*10000


In [ ]:
geo_data = gpd.read_file('/content/drive/My Drive/s_boundary.geojson', encoding='cp949')
geo_data.columns = ['adm_nm', 'adm_cd', '행정동코드', 'sidonm', 'sggnm', '행정동', '위도', '경도',
       'geometry']
geo_data

,adm_nm,adm_cd,행정동코드,sidonm,sggnm,행정동,위도,경도,geometry
0,서울특별시 종로구 사직동,1101053,1111053000,서울특별시,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,서울특별시 종로구 삼청동,1101054,1111054000,서울특별시,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,서울특별시 종로구 부암동,1101055,1111055000,서울특별시,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,서울특별시 종로구 평창동,1101056,1111056000,서울특별시,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,서울특별시 종로구 무악동,1101057,1111057000,서울특별시,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...,...,...,...
420,서울특별시 강동구 둔촌2동,1125071,1174070000,서울특별시,강동구,둔촌2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ..."
421,서울특별시 강동구 암사1동,1125072,1174057000,서울특별시,강동구,암사1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ..."
422,서울특별시 강동구 천호2동,1125073,1174061000,서울특별시,강동구,천호2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ..."
423,서울특별시 강동구 길동,1125074,1174068500,서울특별시,강동구,길동,37.539725,127.145907,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ..."


In [ ]:
df[df['행정동코드']=='1153078000']

,행정동코드,행정동,시군구명,5-14세,키움센터_개수,초등돌봄교실_교실개수,지역아동센터_개수,방과후아카데미_개수,총시설수,시설tfidf,예술학원,보습학원개수,총학원수,학원tfidf,인프라지수,인프라지수_mmscaler,학원지수_mmscaler,시설지수_mmscaler,학원100,시설100
1,1153078000,오류제2동,구로구,4507,1.0,17.0,2.0,0.0,20.0,0.000062,44,26.0,70.0,0.000037,1.663341,0.00215,0.053983,0.000403,0.371978,0.618727


In [ ]:
# df.loc[424] = ['1153080000','항동', 4507,0.146903,147.529763]
df.loc[424] = lst
# lst =['1153080000','항동', '구로구',4507,1,17,2,0,20,0.000062,44,26,70,0.000037,1.663341,0.00215,0.053983	,0.000403	,0.371978,	0.618727]
# len(lst)

In [ ]:
df.shape

(425, 20)

In [ ]:
pt = np.percentile(df[['학원tfidf']],[i*10 for i in range(1,10)]).tolist()
df['학원percentile'] = 0
df.loc[(df['학원tfidf'] <= pt[0]),'학원percentile'] = '1'
df.loc[(df['학원tfidf'] > pt[0]) & (df['학원tfidf'] <= pt[1]),'학원percentile'] = '2'
df.loc[(df['학원tfidf'] > pt[1]) & (df['학원tfidf'] <= pt[2]),'학원percentile'] = '3'
df.loc[(df['학원tfidf'] > pt[2]) & (df['학원tfidf'] <= pt[3]),'학원percentile'] = '4'
df.loc[(df['학원tfidf'] > pt[3]) & (df['학원tfidf'] <= pt[4]),'학원percentile'] = '5'
df.loc[(df['학원tfidf'] > pt[4]) & (df['학원tfidf'] <= pt[5]),'학원percentile'] = '6'
df.loc[(df['학원tfidf'] > pt[5]) & (df['학원tfidf'] <= pt[6]),'학원percentile'] = '7'
df.loc[(df['학원tfidf'] > pt[6]) & (df['학원tfidf'] <= pt[7]),'학원percentile'] = '8'
df.loc[(df['학원tfidf'] > pt[7]) & (df['학원tfidf'] <= pt[8]),'학원percentile'] = '9'
df.loc[(df['학원tfidf'] > pt[8]),'학원percentile'] = '10'

In [ ]:
# len(df[df['학원percentile']=='10'])
# len(df[df['학원percentile']=='1'])
pt

[2.117800513196267e-05,
 2.7858434504011733e-05,
 3.354908103830883e-05,
 3.789881293946471e-05,
 4.179249886158735e-05,
 4.684562209915407e-05,
 5.232321125705179e-05,
 5.809142867332498e-05,
 7.229390027787038e-05]

In [ ]:
pt = np.percentile(df[['시설tfidf']],[i*10 for i in range(1,10)]).tolist()
df['시설percentile'] = 0
df.loc[(df['시설tfidf'] <= pt[0]),'시설percentile'] = '1'
df.loc[(df['시설tfidf'] > pt[0]) & (df['시설tfidf'] <= pt[1]),'시설percentile'] = '2'
df.loc[(df['시설tfidf'] > pt[1]) & (df['시설tfidf'] <= pt[2]),'시설percentile'] = '3'
df.loc[(df['시설tfidf'] > pt[2]) & (df['시설tfidf'] <= pt[3]),'시설percentile'] = '4'
df.loc[(df['시설tfidf'] > pt[3]) & (df['시설tfidf'] <= pt[4]),'시설percentile'] = '5'
df.loc[(df['시설tfidf'] > pt[4]) & (df['시설tfidf'] <= pt[5]),'시설percentile'] = '6'
df.loc[(df['시설tfidf'] > pt[5]) & (df['시설tfidf'] <= pt[6]),'시설percentile'] = '7'
df.loc[(df['시설tfidf'] > pt[6]) & (df['시설tfidf'] <= pt[7]),'시설percentile'] = '8'
df.loc[(df['시설tfidf'] > pt[7]) & (df['시설tfidf'] <= pt[8]),'시설percentile'] = '9'
df.loc[(df['시설tfidf'] > pt[8]),'시설percentile'] = '10'

In [ ]:
df

,행정동코드,행정동,시군구명,5-14세,키움센터_개수,초등돌봄교실_교실개수,지역아동센터_개수,방과후아카데미_개수,총시설수,시설tfidf,예술학원,보습학원개수,총학원수,학원tfidf,인프라지수,인프라지수_mmscaler,학원지수_mmscaler,시설지수_mmscaler,학원100,시설100,학원percentile,시설percentile
0,1165065100,양재1동,서초구,5145,1.0,21.0,2.0,0.0,24.0,0.000011,35,17.0,52.0,0.000024,0.445650,0.000000,0.033913,0.000000,0.243241,0.108400,2,1
1,1153078000,오류제2동,구로구,4507,1.0,17.0,2.0,0.0,20.0,0.000062,44,26.0,70.0,0.000037,1.663341,0.002150,0.053983,0.000403,0.371978,0.618727,4,1
2,1150063000,방화제1동,강서구,4970,1.0,12.0,3.0,0.0,16.0,0.000101,42,47.0,89.0,0.000043,2.361944,0.003383,0.062654,0.000711,0.427595,1.009956,6,1
3,1138069000,진관동,은평구,7384,1.0,21.0,1.0,0.0,23.0,0.000015,37,27.0,64.0,0.000021,0.726752,0.000496,0.028396,0.000034,0.207859,0.151062,1,1
4,1138062500,역촌동,은평구,3381,1.0,6.0,4.0,0.0,11.0,0.000231,28,43.0,71.0,0.000050,4.592669,0.007321,0.074386,0.001737,0.502845,2.309401,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,1114059000,광희동,중구,170,0.0,0.0,0.0,0.0,0.0,0.008202,1,0.0,1.0,0.000028,295.243036,0.520456,0.039300,0.064626,0.277797,82.017688,2,10
421,1111054000,삼청동,종로구,165,0.0,0.0,0.0,0.0,0.0,0.008450,1,0.0,1.0,0.000029,295.243036,0.520456,0.040612,0.066587,0.286215,84.503073,3,10
422,1114060500,을지로동,중구,44,0.0,0.0,0.0,0.0,0.0,0.031689,2,0.0,2.0,0.000161,196.828690,0.346709,0.246992,0.249936,1.609961,316.886522,10,10
423,1174069000,둔촌제1동,강동구,11,0.0,0.0,0.0,0.0,0.0,0.126755,2,0.0,2.0,0.000644,196.828690,0.346709,1.000000,1.000000,6.439844,1267.546089,10,10


### 인프라지수 시각화(folium)

In [ ]:
geo_data

,adm_nm,adm_cd,행정동코드,sidonm,sggnm,행정동,위도,경도,geometry
0,서울특별시 종로구 사직동,1101053,1111053000,서울특별시,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,서울특별시 종로구 삼청동,1101054,1111054000,서울특별시,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,서울특별시 종로구 부암동,1101055,1111055000,서울특별시,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,서울특별시 종로구 평창동,1101056,1111056000,서울특별시,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,서울특별시 종로구 무악동,1101057,1111057000,서울특별시,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...,...,...,...
420,서울특별시 강동구 둔촌2동,1125071,1174070000,서울특별시,강동구,둔촌2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ..."
421,서울특별시 강동구 암사1동,1125072,1174057000,서울특별시,강동구,암사1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ..."
422,서울특별시 강동구 천호2동,1125073,1174061000,서울특별시,강동구,천호2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ..."
423,서울특별시 강동구 길동,1125074,1174068500,서울특별시,강동구,길동,37.539725,127.145907,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ..."


In [ ]:
center = [37.541, 126.986]
# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다.
m11 = folium.Map(location=center, zoom_start=10)

# Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(
    geo_data=geo_data,
    data=df,
    columns=('행정동코드', '시설percentile'),
    key_on='feature.properties.행정동코드',
    fill_color='BuPu',
    legend_name='시설',
).add_to(m11)
m11
m.save('시설중요도_percentile.html')

In [ ]:
center = [37.541, 126.986]
# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듭니다.
m2 = folium.Map(location=center, zoom_start=10)

# Choropleth 레이어를 만들고, 맵 m에 추가합니다.
folium.Choropleth(
    geo_data=geo_data,
    data=df,
    columns=('행정동코드', '학원percentile'),
    key_on='feature.properties.행정동코드',
    fill_color='BuPu',
    legend_name='학원중요도',
).add_to(m2)
m2
m2.save('학원중요도_percentile.html')